### Import modules

In [2]:
import tensorflow as tf

# from https://medium.com/ibm-data-ai/memory-hygiene-with-tensorflow-during-model-training-and-deployment-for-inference-45cf49a15688
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(str(gpu))
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])


import tensorflow.keras as keras
from keras import layers
import numpy as np
from dataset import *

2023-01-03 21:28:52.903490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 21:28:53.624450: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-03 21:28:54.737487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-03 21:28:54.737652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


#### GPU config

### Create dataset from test audio file


In [ ]:
xf, X, spec_params = create_dataset('audio/training_audio.wav', 2, 0.5)
X = np.abs(X)
print(X.shape)

## Define model

In [9]:
inputs = keras.Input((256, 1024))

# Encoder - Conv1D gradually down to LSTM
filter_expand = inputs

for i in range(7):
    conv = layers.Conv1D(2**i, 3, 1, padding='same')(filter_expand)
    add = layers.Add()([filter_expand, conv])
    conv = layers.Conv1D(2**i, 3, 1, padding='same')(add)
    add = layers.Add()([add, conv])
    pool = layers.MaxPool1D(2, 2, padding='valid')(add)
    filter_expand = layers.Conv1D(2**(i+1), 3, 1, padding='same')(pool)



outputs = filter_expand

model = keras.Model(inputs=inputs, outputs=outputs, name="conv1d-lstm")

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "conv1d-lstm"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 1024)]  0           []                               
                                                                                                  
 conv1d_24 (Conv1D)             (None, 256, 1)       3073        ['input_4[0][0]']                
                                                                                                  
 add_17 (Add)                   (None, 256, 1024)    0           ['input_4[0][0]',                
                                                                  'conv1d_24[0][0]']              
                                                                                                  
 conv1d_25 (Conv1D)             (None, 256, 1)       3073        ['add_17[0][0]']       

In [ ]:
X_fit = tf.expand_dims(X, -1)
X_fit = (X_fit - np.min(X_fit))/(np.max(X_fit) - np.min(X_fit))

print("==================")
print(X_fit.shape)
print("==================")


In [ ]:
model.load_weights("cnn_autoencoder")

In [ ]:
model.fit(X_fit, X_fit,
        epochs=200,
        shuffle=True
)

### COMPARE SPECTROGRAMS IN IMAGES


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x_example = X[0]

plt.imsave("INPUT_EXAMPLE.png", x_example)
x_example = np.reshape(x_example, (1, x_example.shape[0], x_example.shape[1], 1))
print(x_example.shape)
prediction = model.predict(x_example)
plt.imsave("OUTPUT_EXAMPLE.png", prediction[0, :, :, 0])

In [ ]:
model.save("cnn_autoencoder")

### Save to audio file

In [ ]:
import importlib
import postprocessing

In [ ]:
importlib.reload(postprocessing)
out_samp, out_win, out_stride = spec_params
audio = postprocessing.reverse_spectrogram(prediction[0, :, :, 0].transpose(), out_samp, out_win, out_stride)
audio = np.reshape(audio, (-1, 1))
f_out = open_write("test_output_cnn.wav", 1, 2, 44100)
write(f_out, audio)